# Catenable Lists

The class of *catenable lists* is an extension of *output-restricted deques* with an efficient catenation operator `++`.

|    instance    | persistence | amortization | empty | isEmpty |  cons  |  snoc  |   ++  | head |     tail     |
|:--------------:|:-----------:|:------------:|:-----:|:-------:|:------:|:------:|:-----:|:----:|:------------:|
| Catenable List |  persistent |      yes     |  O(1) |   O(1)  | O(1)** | O(1)** | O(1)* | O(1) | O(n) / O(1)* |

** amortized time*

*** depends on the complexity of `snoc` of the underlying queue*

## Queue Bootstrapping
Since the only instance `CatList` of the `CatenableList` class that is presented here is based on *structural abstraction*, the code sample below includes the definition of an ordinary `Queue`. This *primitive* queue can be an arbitrary persistent FIFO queue with either amortized or worst-case $O(1)$ operations.

In [ ]:
class Queue q where
    
    empty :: q a
    isEmpty :: q a -> Bool
    
    snoc :: q a -> a -> q a
    
    head :: q a -> a
    tail :: q a -> q a

In [ ]:
class CatenableList c where

    empty :: c a
    isEmpty :: c a -> Bool
    
    cons :: a -> c a -> c a
    snoc :: c a -> a -> c a
    (++) :: c a -> c a -> c a
    
    head :: c a -> a
    tail :: c a -> c a

## List with Efficient Catenation
`CatList` uses *structural abstraction* technique to bootstrap a catenable list instance from an efficient persistent FIFO queue.

### Representation
The `CatList` is generic not only over the type of elements `a` it contains but also over the type of primitive queue `q` it bootstraps from.

There are two constructors:
 1. `E` represents an empty list
 1. `C a (q ...)` is a non-uniform constructor of a non-empy list containing single element `a` and a primitive queue with elements being catenable lists, i.e. `q (CatList q a)`

One can interpret this non-uniform type as a tree with nodes `C` containing elements `a` and trivial leafs `E`.  In this interpretation the children of a node are stored in a queue `q` in a left-to-right order (preorder).

### Operations
Because elements are stored in a preorder traversal of the tree from the interpretation mentioned above, `head` can be trivially obtained from the root and thus runs in $O(1)$ worst-case time.

The main operation in this class is the catenation operator `(++)` which, due to the structural abstraction from FIFO queues, reduces to adding (in this case `link`ing) one list as an element of the primitive queue of the other. Since we assumed an efficient `Queue` instance, `(++)` runs in either $O(1)$ worst-case or amortized time.

Similarly to `(++)`, `cons` and `snoc` just link the list together with a new singleton list (or the other way).

Finally, `tail` (i.e. removing the head element) is the most complex operation. In the tree interpretation, if the root gets discarded, then `tail` must link all the the children to form a new `CatList`. 
 - This `linkAll` operation first extracts the first child tree `t` and the rest `q'` with `head` and `tail` on the list of children `q` and then `link`s `t` with recursively linked trees in `q'`.
 - Splitting the children clearly depeneds on `head` and `tail` of the `Queue` (assumed to be $O(1)$) and since each recursive call to `linkAll` is suspended and forced when a tree is removed from the queue, the overall amortized complexity is $O(1)$.

### Benefits
> Given a good implementation of queues, this is the fastest known implementation of persistent catenable lists, especially for applications that use persistence heavily.

In [ ]:
-- | Catenable list which can be interpreted as a tree with elements in nodes 'C' and trivial leafs 'E'
data CatList q a = E | C a (q (CatList q a))

-- | Link two lists together in O(1) time.
-- |
-- | Linking is done by adding the second list, interpreted as a tree as the last child of the first one.
-- |
-- | Note: The time complexity actually depends on the 'snoc' operation of the primitive 'Queue' instance.
link :: Queue q => CatList q a -> CatList q a -> CatList q a
link (C x q) s = C x (Ghci2.snoc q s)

instance Queue q => CatenableList (CatList q) where

    -- | Construct new (empty) list in O(1) worst-case time
    empty = E
    
    -- | Check whether a list is empty in O(1) worst-case time
    isEmpty E = True
    isEmpty _ = False
    
    -- | Prepend new item to the front of a list in O(1) worst-case or amortized time depending on 'q'
    cons x xs = C x Ghci2.empty ++ xs
    
    -- | Append new item to the back of a list in O(1) worst-case or amortized time depending on 'q'
    snoc xs x = xs ++ C x Ghci2.empty
    
    -- | Append two lists together in O(1) amortized time.
    -- |
    -- | Note: The complexity depends on 'link', resp. the 'q' instance and amortization comes from
    -- |       the potential operation sequencing of '++' and 'tail' (which is amortized).
    xs ++ E = xs
    E ++ xs = xs
    xs ++ ys = link xs ys
    
    -- | Retrieve the head item of a non-empty list in O(1) worst-case time
    head E = error "List is empty"
    head (C x _) = x
    
    -- | Remove the head of a non-empty list and retrieve the rear in O(1) amortized time.
    -- |
    -- | Note: The time complexity assumes efficient 'head', 'tail' and 'snoc' implementations on 'q'.
    tail E = error "List is empty"
    tail (C x q) = if Ghci2.isEmpty q then E else linkAll q
        where
            linkAll q = if Ghci2.isEmpty q' then t else link t (linkAll q')
                where
                    t = Ghci2.head q
                    q' = Ghci2.tail q